In [1]:
#!usr/bin/python
import numpy as np
import itertools
import HFStability

#Each run is an instance of PyHEG
run = HFStability.PyHEG()

#input params
run.rs = 1.
run.ndim = 3
run.Nk = 10

#Determine other parameters
if run.ndim == 1:
    run.kf = np.pi / (4 *run.rs)
elif run.ndim == 2:
    run.kf = 2**0.5 /run.rs
elif run.ndim == 3:
    run.kf = (9 * np.pi / 4)**(1./3.) * (1 / run.rs) 

kmax = 2.0 * run.kf
kgrid = np.linspace(-kmax, kmax, run.Nk)
run.bzone_length = np.pi / kmax
run.vol = run.bzone_length**(run.ndim)

temp = range(run.Nk)
states = temp
for i in range(run.ndim - 1):
    states = itertools.product(states,temp)
states = list(states)

#unpack into list of tuples
if run.ndim == 3:
    states = [(item[0][0], item [0][1], item[1]) for item in states]

if run.ndim == 1:
    states = [[kgrid[state]] for state in states]
else:
    states = [[kgrid[i] for i in state] for state in states]

occ_states = []
vir_states = []
for index, state in enumerate(states):
    k = np.linalg.norm(state)
    if k < run.kf + 10e-4:
        occ_states.append(index)
    else:
        vir_states.append(index)

run.Nocc = len(occ_states)
run.Nvir = len(vir_states)
run.states = np.asarray(states)

#RHF ONLY
run.N_elec = 2*run.Nocc
print run.states

[[-3.83831659 -3.83831659 -3.83831659]
 [-3.83831659 -3.83831659 -2.98535734]
 [-3.83831659 -3.83831659 -2.1323981 ]
 ..., 
 [ 3.83831659  3.83831659  2.1323981 ]
 [ 3.83831659  3.83831659  2.98535734]
 [ 3.83831659  3.83831659  3.83831659]]


In [7]:
excitations = np.random.rand((3,5), dtype=np.uint32)
for i in range(3):
    for j in range(5):
        excitations[i,j] = 10*i + j


print excitations
run.excitations = excitations
print run.excitations

TypeError: rand() got an unexpected keyword argument 'dtype'